In [7]:

import os
import random


def find_similar_characters(word1, word2):  # find the similar characters between the words
    similar_positions = []

    for i in range(len(word1)):
        for j in range(len(word2)):
            if word1[i] == word2[j]:
                similar_positions.append((i, j))

    return similar_positions


def create_similarity_dictionary(
        word_list):  # create a dicitonary that has for each two words the indices of similar letters
    similarity_dict = {}

    for i in range(len(word_list)):
        for j in range(len(word_list)):
            word1 = word_list[i]
            word2 = word_list[j]
            similarity_key = (i, j)

            similar_positions = find_similar_characters(word1, word2)

            if similarity_key not in similarity_dict:
                similarity_dict[similarity_key] = []

            similarity_dict[similarity_key].extend(similar_positions)

    return similarity_dict


def display_words_in_grid(random_array, words):  # show the grid for a particular individual
    # Create an empty 20x20 grid
    grid = np.full((20, 20), '.', dtype=str)

    for i in range(len(random_array)):
        word = words[i]
        x, y, orientation = random_array[i]

        for j, letter in enumerate(word):
            if orientation == 0:  # Horizontal
                grid[(y + j) % 20, x] = letter
            else:  # Vertical
                grid[y, (x + j) % 20] = letter

    # Display the grid
    for row in grid:
        print(' '.join(row))


def overlapping_length(range1, range2):  # get the overlapping length of two ranges
    start1, end1 = range1
    start2, end2 = range2

    # print(start1,end1,start2,end2,end12,end22)
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)

    # print(overlap_end2)
    overlap_sum = 0
    if overlap_start < overlap_end:
        overlap_sum += (overlap_end - overlap_start)
        # print(overlap_sum)
    return overlap_sum


def get_num_of_parallel_chars(words, array_0, array_1, array_2, zeros, ones):  # get the num of parallel chars
    sum = 0
    for i in zeros:
        for indexj, j in enumerate(array_1):
            if indexj != i and j in range(array_1[i], array_1[i] + len(words[i])) and abs(
                    array_2[indexj] - array_2[i]) < 2 and array_0[indexj] != 1 and array_1[indexj] != -1:
                zeros.remove(indexj)
                range2 = (array_1[i], (array_1[i] + len(words[i])))
                range1 = (array_1[indexj], (array_1[indexj] + len(words[indexj])))
                sum += overlapping_length(range1, range2)
                array_1[indexj] = -1
    for i in ones:
        for indexj, j in enumerate(array_2):
            if indexj != i and j in range(array_2[i], array_2[i] + len(words[i])) and abs(
                    array_1[indexj] - array_1[i]) < 2 and array_0[indexj] != 0 and array_2[indexj] != -1:
                ones.remove(indexj)
                range2 = (array_2[i], (array_2[i] + len(words[i])))
                range1 = (array_2[indexj], (array_2[indexj] + len(words[indexj])))
                sum += overlapping_length(range1, range2)
                array_2[indexj] = -1
    return sum


def check_crossover_pair(grid, word1, word2, length1, length2, text1, text2,
                         intersections):  # check if two words are crossing correctly
    if word1[2]:
        if (word1[2] != word2[2] and word2[0] in range(word1[0], word1[0] + length1) and word1[1] in range(word2[1],
                                                                                                           word2[
                                                                                                               1] + length2)):
            if (grid[word2[0]][word1[1]] == text1[word2[0] - word1[0]] and grid[word2[0]][word1[1]] == text2[
                word1[1] - word2[1]]):
                intersections[word2[0]][word1[1]] = 2
                return True
    else:
        if (word1[2] != word2[2] and word1[0] in range(word2[0], word2[0] + length2) and word2[1] in range(word1[1],
                                                                                                           word1[
                                                                                                               1] + length1)):
            if (grid[word1[0]][word2[1]] == text2[word1[0] - word2[0]] and grid[word1[0]][word2[1]] == text1[
                word2[1] - word1[1]]):
                intersections[word1[0]][word2[1]] = 2
                return True
    return False


edge = []
visited = []
group_size = 0
indices = []
largest_indices = []


def dfs_cop(node):  # dfs
    global group_size, indices
    if visited[node]:
        return
    visited[node] = 1
    indices.append(node)
    group_size += 1
    for i in edge:
        if (i[0] == node):
            dfs_cop(i[1])
        if (i[1] == node):
            dfs_cop(i[0])


def count_num_cop(nodes, words, grid,
                  intersections):  # count the number of words in the largest component (group) of correctly intersecting words
    global edge, visited, group_size, largest_indices, indices
    visited = [0 for i in range(len(nodes))]
    edge = []
    for idx, i in enumerate(nodes):
        for j in range(idx + 1, len(nodes)):
            if (check_crossover_pair(grid, i, nodes[j], len(words[idx]), len(words[j]), words[idx], words[j],
                                     intersections)):
                # print(words[idx],words[j],i[0],i[1],nodes[j][0],nodes[j][1])
                edge.append((idx, j))
    res = 0
    res2 = 0
    for i in range(len(nodes)):
        if visited[i]:
            continue
        group_size = 0
        res2 += 1
        indices = []

        dfs_cop(i)
        if (res < group_size):
            res = group_size
            largest_indices = indices.copy()
            # print("yes")
            # print(indices)

    return (res, res2)


def get_fitness(random_array, words, printer=0):  # calculate the fitness
    grid_size = 20

    # Create a 2D list filled with zeros
    intersections = [[0 for _ in range(grid_size)] for _ in range(grid_size)]

    # Create a 2D list filled with dots
    grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]

    # print(random_array)
    wrong_intersections = 0
    correct_intersections = 0
    for i in range(len(random_array)):
        word = words[i % len(words)]
        x, y, orientation = random_array[i]
        # print(word)
        for j, letter in enumerate(word):
            if orientation:
                # print(((y + j)%20),x)
                intersections[x + j][y] = 1
                # grid[x, (y + j)%20] = letter
                if grid[x + j][y] == '.':
                    grid[x + j][y] = letter
                elif letter == grid[x + j][y]:
                    correct_intersections += 1
                else:
                    wrong_intersections += 1
                grid[x + j][y] = letter
            else:
                intersections[x][y + j] = 1
                # grid[(x + j)%20, y] = letter
                if grid[x][y + j] == '.':
                    grid[x][y + j] = letter
                elif letter == grid[x][y + j]:
                    correct_intersections += 1
                else:
                    wrong_intersections += 1
                grid[x][y + j] = letter

    array_0 = []
    array_1 = []
    array_2 = []

    for index, i in enumerate(random_array):
        array_2.append(i[0])
        array_1.append(i[1])
        array_0.append(i[2])

    zeros = []
    ones = []
    for index, i in enumerate(array_0):
        if i == 0:
            zeros.append(index)
            # print(index, "0?")
        else:
            ones.append(index)

    fitness = 0
    fitness += get_num_of_parallel_chars(words, array_0, array_1, array_2, zeros, ones) * 100

    out = count_num_cop(random_array, words, grid, intersections)

    fitness += (len(words) - out[0]) * 10  # +((out[1])-1)*1
    fitness += wrong_intersections * 10000
    if printer:
        print("number of wrong intersections", wrong_intersections)
    return fitness


def get_fitness_short(random_array, words, printer=0):  # fast version of fitness function to be used in mutate
    grid_size = 20

    # Create a 2D list filled with zeros
    intersections = [[0 for _ in range(grid_size)] for _ in range(grid_size)]

    # Create a 2D list filled with dots
    grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]

    # print(random_array)
    wrong_intersections = 0
    correct_intersections = 0
    for i in range(len(random_array)):
        word = words[i % len(words)]
        x, y, orientation = random_array[i]
        # print(word)
        for j, letter in enumerate(word):
            if orientation:
                intersections[x + j][y] = 1
                if grid[x + j][y] == '.':
                    grid[x + j][y] = letter
                elif letter == grid[x + j][y]:
                    correct_intersections += 1
                else:
                    wrong_intersections += 1
                grid[x + j][y] = letter
            else:
                intersections[x][y + j] = 1
                # grid[(x + j)%20, y] = letter
                if grid[x][y + j] == '.':
                    grid[x][y + j] = letter
                elif letter == grid[x][y + j]:
                    correct_intersections += 1
                else:
                    wrong_intersections += 1
                grid[x][y + j] = letter

    fitness = 0
    out = count_num_cop(random_array, words, grid, intersections)

    fitness += (len(words) - out[0]) * 10  # +((out[1])-1)*1
    fitness += wrong_intersections * 10000
    if printer:
        print("number of wrong intersections", wrong_intersections)
    return fitness


def display_words_in_grid(random_array, words):  # show the grid of an individual
    # Create an empty 20x20 grid
    grid_size = 20

    # Create a 2D list filled with zeros

    # Create a 2D list filled with dots
    grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]

    for i in range(len(random_array)):
        word = words[i]
        x, y, orientation = random_array[i]
        # print(word)
        # print( random_array[i])
        for j, letter in enumerate(word):
            # print(letter, j)
            if orientation:  # Horizontal
                grid[x + j][y] = letter
            else:  # Vertical
                grid[x][y + j] = letter
    for row in grid:
        print(' '.join(row))


def get_individual(n, s=20):  # create a random individual
    array_size = 3 * n

    # Creating a random array with shape (array_size,)
    random_array = [random.randint(0, s - 1) for _ in range(array_size)]

    # Reshaping the array to have shape (n, 3)
    random_array = [random_array[i:i + 3] for i in range(0, array_size, 3)]

    for index, i in enumerate(random_array):
        i[2] = i[2] % 2
        if not i[2]:
            i[1] = i[1] % max(1, (s - len(words[index]) + 1))
        else:
            i[0] = i[0] % max(1, (s - len(words[index]) + 1))

    return random_array


def initial_population(n_words, population_size):  # intialize the population
    result_dict = create_similarity_dictionary(words)
    print(result_dict)
    population = [get_individual(n_words) for _ in range(population_size)]
    population.sort(key=lambda x: get_fitness(x, words))
    return population


def get_parents(population, n_offsprings):  # get the parents to breed
    print(n_offsprings)
    print("len pop", len(population))
    # here we take the most fit parents for crossover
    # print(population)
    random_elements = random.sample(population, 25)

    mutated = []

    mutated = [mutate(population[0].copy()) for _ in range(3)]
    print(get_fitness(population[0], words))
    for i in mutated:
        print(get_fitness(i, words))
    mothers = (population[1:n_offsprings:2] + random_elements + mutated).copy()
    mutated = [mutate(population[0].copy()) for _ in range(3)]

    print("motherL ", len(mothers))
    random_elements = random.sample(population, 25)

    print(len(mothers))
    fathers = (population[:n_offsprings:2] + random_elements + mutated).copy()
    return mothers, fathers


import copy
import random


def cross(array1, array2):  # cross two parents
    global largest_indices  # Assuming this variable is defined outside the function

    # Assuming array1 and array2 have the same shape
    rows, cols = len(array1), len(array1[0])

    final = [copy.deepcopy(array1), copy.deepcopy(array2)]
    children = []
    result_array1 = [[0] * cols for _ in range(rows)]
    result_array2 = [[0] * cols for _ in range(rows)]

    get_fitness_short(array1, words)
    indices1 = list(largest_indices)
    get_fitness_short(array2, words)
    indices2 = list(largest_indices)

    for _ in range(3):
        for i in range(rows):
            # Choose randomly between array1 and array2 for each element
            if i in indices1:
                result_array1[i] = final[0][i].copy()
            else:
                result_array1[i] = final[1][i].copy()

            if i in indices2:
                result_array2[i] = final[1][i].copy()
            else:
                result_array2[i] = final[0][i].copy()

        child1 = copy.deepcopy(result_array1)
        child2 = copy.deepcopy(result_array2)
        children.append(child1)
        children.append(child2)

    return children


def mutate(offspring):  # mutate an individual as described in the report
    get_fitness_short(offspring, words)
    new_indi = get_individual(len(offspring))
    inds = []
    for i in range(len(words)):
        if not (i in largest_indices):
            inds.append(i)
    # print(inds)

    if len(inds) > 0 and len(largest_indices) > 1:
        rnum = random.randint(0, len(inds) - 1)
        char_nums = tuple()
        my_choice_list = []
        for choice in largest_indices:
            if (len(result_dict[(inds[rnum], choice)])):
                my_choice_list.append(choice)
                break
        if len(my_choice_list) == 0:
            return offspring
        my_choice = random.choice(my_choice_list)
        # my_choice = random.choice(largest_indices)
        char_nums = random.choice(result_dict[(inds[rnum], my_choice)])
        offspring[inds[rnum]] = offspring[my_choice].copy()
        offspring[inds[rnum]][2] = 1 - offspring[inds[rnum]][2]
        if (offspring[inds[rnum]][2]):
            offspring[inds[rnum]][1] += char_nums[1]
            offspring[inds[rnum]][0] -= char_nums[0]
            if offspring[inds[rnum]][0] < 0:
                offspring[inds[rnum]][0] = 0
            if offspring[inds[rnum]][0] + len(words[inds[rnum]]) > 20:
                offspring[inds[rnum]][0] = 20 - len(words[inds[rnum]])
        else:
            offspring[inds[rnum]][0] += char_nums[1]
            offspring[inds[rnum]][1] -= char_nums[0]
            if offspring[inds[rnum]][1] < 0:
                offspring[inds[rnum]][1] = 0
            if offspring[inds[rnum]][1] + len(words[inds[rnum]]) > 20:
                offspring[inds[rnum]][1] = 20 - len(words[inds[rnum]])
        # display_words_in_grid(offspring,words)
    else:
        # print("2")
        num = random.randint(0, 2)
        for index in range(num):
            ind = random.randint(0, len(words) - 1)
            offspring[ind] = new_indi[ind]

    return offspring  # replace "pass" with "return"


def replace_population(population, new_individuals):  # replace the old popultion with a new one
    # replace least fit individuals with better ones
    size = len(population)
    # new_generated_individuals = [get_individual(len(words)) for _ in range(int(size / 5))]
    final = (population[:int(size / 5)] + new_individuals[
                                          :4 * int(size / 5)]).copy()  # + new_generated_individuals).copy()
    # final.sort(key=lambda x: get_fitness(x, words))
    fitness = [get_fitness(x, words) for x in final]
    fitness.sort()
    final.sort(key=lambda x: get_fitness(x, words))
    copies_of_the_best = [mutate(final[0].copy()) for _ in range(35)]
    for sol in copies_of_the_best:
        if (get_fitness(sol, words) < get_fitness(final[0], words)):
            print("one")
            print(get_fitness(final[0], words), "----->", get_fitness(sol, words))
            final.insert(0, sol)
            del final[len(final) - 1]
        else:
            print(get_fitness(sol, words))
    copies_of_the_best = [mutate(final[3].copy()) for _ in range(20)]
    for sol in copies_of_the_best:
        if (get_fitness(sol, words) < get_fitness(final[0], words)):
            print("one")
            print(get_fitness(final[0], words), "----->", get_fitness(sol, words))
            final.insert(0, sol)
            del final[len(final) - 1]
        else:
            print(get_fitness(sol, words))
    copies_of_the_best = [mutate(final[5].copy()) for _ in range(15)]
    for sol in copies_of_the_best:
        if (get_fitness(sol, words) < get_fitness(final[0], words)):
            print("one")
            print(get_fitness(final[0], words), "----->", get_fitness(sol, words))
            final.insert(0, sol)
            del final[len(final) - 1]
        else:
            print(get_fitness(sol, words))
    copies_of_the_best = [mutate(final[10].copy()) for _ in range(15)]
    for sol in copies_of_the_best:
        if (get_fitness(sol, words) < get_fitness(final[0], words)):
            print("one")
            print(get_fitness(final[0], words), "----->", get_fitness(sol, words))
            final.insert(0, sol)
            del final[len(final) - 1]
        else:
            print(get_fitness(sol, words))
    return final  # replace "pass" with "return"
    ############################################################################


def evolution_step(population, population_size, n_offsprings):  # start the evolution step
    mothers, fathers = get_parents(population, n_offsprings)
    offsprings = []
    print("len mothers", len(mothers))
    for _ in range(3):
        random.shuffle(mothers)
        for mother, father in zip(mothers, fathers):
            offsprings_temp = cross(mother, father)
            for off in offsprings_temp:
                offsprings.append(off)

    offsprings = [mutate(x) for x in offsprings]
    offsprings.sort(key=lambda x: get_fitness(x, words))
    new_population = replace_population(population, offsprings)
    print("replaced pop")
    return new_population


def evolution(n_words, population_size, n_offsprings, generations):
    population = initial_population(n_words, population_size)
    print(population[0])
    display_words_in_grid(population[0], words)
    print("")
    fitness_change = []
    count_to_print = 0
    for generation in range(generations):
        print("generation:", generation)
        population = evolution_step(population, population_size, n_offsprings)
        best_individual = population[0]
        print("the best:", get_fitness(best_individual, words))

        count_to_print += 1
        if (count_to_print == 1):
            count_to_print = 0

            indi = population[-1]
            display_words_in_grid(indi, words)
            print()
            print()
            display_words_in_grid(best_individual, words)
            for _ in range(10):
                print(get_fitness(population[_], words), end="  ")
            print("")
            print("worst", get_fitness(indi, words))
            print("median", get_fitness(population[int(len(population) / 2)], words))
            bst = get_fitness(best_individual, words, printer=1)
            print("the best:", bst)
            print("static", largest_indices)
            print("number of words in the largest group", len(largest_indices))
        fitness_change.append(get_fitness(best_individual, words))
        if bst == 0:  # stop if a solution is found
            break

    display_words_in_grid(best_individual, words)
    print(best_individual)
    print(get_fitness(best_individual, words))
    display_words_in_grid(indi, words)
    print(get_fitness(indi, words))
    sum = 0
    for ind in population:
        sum += get_fitness(ind, words)

    avg = sum / len(population)
    return fitness_change, get_fitness(indi, words), avg, best_individual







    # Split the content into a list of words

print(words)
result_dict = create_similarity_dictionary(words)
print(result_dict)
generations = 2000
fitness_change, maks, avg, best_individual = evolution(len(words), population_size=250, n_offsprings=180,
                                                        generations=generations)


['cage', 'cemetery', 'chemistry', 'engine', 'fairytale', 'gate', 'pillow', 'train', 'widow', 'wine']
{(0, 0): [(0, 0), (1, 1), (2, 2), (3, 3)], (0, 1): [(0, 0), (3, 1), (3, 3), (3, 5)], (0, 2): [(0, 0), (3, 2)], (0, 3): [(2, 2), (3, 0), (3, 5)], (0, 4): [(1, 1), (1, 6), (3, 8)], (0, 5): [(1, 1), (2, 0), (3, 3)], (0, 6): [], (0, 7): [(1, 2)], (0, 8): [], (0, 9): [(3, 3)], (1, 0): [(0, 0), (1, 3), (3, 3), (5, 3)], (1, 1): [(0, 0), (1, 1), (1, 3), (1, 5), (2, 2), (3, 1), (3, 3), (3, 5), (4, 4), (5, 1), (5, 3), (5, 5), (6, 6), (7, 7)], (1, 2): [(0, 0), (1, 2), (2, 3), (3, 2), (4, 6), (5, 2), (6, 7), (7, 8)], (1, 3): [(1, 0), (1, 5), (3, 0), (3, 5), (5, 0), (5, 5)], (1, 4): [(1, 8), (3, 8), (4, 5), (5, 8), (6, 3), (7, 4)], (1, 5): [(1, 3), (3, 3), (4, 2), (5, 3)], (1, 6): [], (1, 7): [(4, 0), (6, 1)], (1, 8): [], (1, 9): [(1, 3), (3, 3), (5, 3)], (2, 0): [(0, 0), (2, 3)], (2, 1): [(0, 0), (2, 1), (2, 3), (2, 5), (3, 2), (6, 4), (7, 6), (8, 7)], (2, 2): [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4